In [157]:
"""
假如历史数据损坏，或需要其它材料的价格，则需要从网站获取全部历史数据，这种情况就要用到此脚本，原因是每一个材料对应网址的总页数不一样，见page_range变量。
一次只能保存一个原材料的数据，可以在已有的excel文件后新增一个sheet并保存，不需要指定日期范围，而需要去网站上确认该原材料的总页数。
推荐每次操作前创建一个备份文件。
"""

'\n假如历史数据损坏，或需要其它材料的价格，则需要从网站获取全部历史数据，这种情况就要用到此脚本，原因是每一个材料对应网址的总页数不一样，见page_range变量。\n'

In [158]:
from urllib.request import urlopen
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
import datetime
from openpyxl import load_workbook
from colorama import Fore, Back, Style #print colored text
from time import sleep

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36 Edg/88.0.705.56',
    'Cookie': '_last_loginuname=viviyanwei; Hm_lvt_e14a1f00255ca6d026803cca20b08490=1607743514; href=https%3A%2F%2Finfo.mybxg.com%2Fm%2F20%2F1210%2F16%2F91D67724C337E319.html; accessId=5d36a9e0-919c-11e9-903c-ab24dbab411b; qimo_xstKeywords_5d36a9e0-919c-11e9-903c-ab24dbab411b=; _login_token=3191c6adf1eb4ce191bcbc278dc29301; _login_mid=2468603; _login_uid=1626926; _login_ip=116.228.72.50; 3191c6adf1eb4ce191bcbc278dc29301=11%3D5%261%3D10; Hm_lpvt_e14a1f00255ca6d026803cca20b08490=1607926822; qimo_seosource_5d36a9e0-919c-11e9-903c-ab24dbab411b=%E5%85%B6%E4%BB%96%E7%BD%91%E7%AB%99; qimo_seokeywords_5d36a9e0-919c-11e9-903c-ab24dbab411b=%E6%9C%AA%E7%9F%A5; pageViewNum=9',
}
# User-Agent获取：打开edge浏览器，在地址栏输入edge://version/ 里面就有user agent的信息
# Cookie获取：用户名密码可以找李忆君(Yvonne)要，打开我的钢铁网首页，先登录，然后获取cookie可以参考这个帖子https://blog.csdn.net/MuWinter/article/details/75313476

In [159]:
# start_date = datetime.date(2021,4,1)
# end_date = datetime.date(2021,4,5)
page_range = 9 #所选原材料网页的总页数
local_path = r'2019-now.xlsx'

"""
dict_urls_1 has table header
dict_urls_2 has no table header
"""
dict_urls = {
    'dict_urls_1': { 
    '无锡300系不锈钢冷轧卷板': 'https://list.info.mybxg.com/market/p-665,666,667,2222,2257,2259------0-01010301-------1.html', #原来叫‘无锡300系冷轧卷板’
    '无锡300系不锈钢热轧卷板': 'https://list.info.mybxg.com/market/p-671-0----0102160102-0-01010301-------1.html', #原来叫‘无锡300系热轧卷板’
    '300系不锈钢工业无缝管': 'https://list.info.mybxg.com/market/p-681-----0102160203-0--------1.html',
    '300系不锈钢工业焊管': 'https://list.info.mybxg.com/market/p-678-----010216020201-0--------1.html',
    '不锈钢角钢': 'https://list.info.mybxg.com/market/p-691-----0102160303-0--------1.html'
    },
    'dict_urls_2': { 
    '上海市场工角槽钢': 'https://list1.mysteel.com/market/p-227-0----01010701-0-010101-------1.html',
    '上海市场冷轧板卷': 'https://list1.mysteel.com/market/p-221-----01010401-0-010101-------1.html',
    '上海市场热轧板卷': 'https://list1.mysteel.com/market/p-221-----01010301-0-010101-------1.html',
    '上海市场低合金卷': 'https://list1.mysteel.com/market/p-221-----01010302-0-010101-------1.html',
    '上海市场镀锌板卷': 'https://list1.mysteel.com/market/p-221-----01010501-0-010101-------1.html',
    '上海市场无缝管': 'https://list1.mysteel.com/market/p-221-----01010901-0-010101-------1.html',
    '沈阳市场H型钢': 'https://list1.mysteel.com/market/p-223-----01010705-0-01050101-------1.html',
    '上海市场方矩管': 'https://list1.mysteel.com/market/p-221-----01010904-0-010101-------1.html',
    '长春市场工角槽钢': 'https://list1.mysteel.com/market/p-223-----01010701-0-01050201-------1.html',
    '长春市场冷轧板卷': 'https://list1.mysteel.com/market/p-223-----01010401-0-01050201-------1.html',
    '烟台市场热轧开平板': 'https://list1.mysteel.com/market/p-223-----01010306-0-01010205-------1.html'
    }
}

dict_urls_1_list = list(dict_urls['dict_urls_1'].items())
dict_urls_2_list = list(dict_urls['dict_urls_2'].items())
dict_urls_2_list

[('上海市场工角槽钢',
  'https://list1.mysteel.com/market/p-227-0----01010701-0-010101-------1.html'),
 ('上海市场冷轧板卷',
  'https://list1.mysteel.com/market/p-221-----01010401-0-010101-------1.html'),
 ('上海市场热轧板卷',
  'https://list1.mysteel.com/market/p-221-----01010301-0-010101-------1.html'),
 ('上海市场低合金卷',
  'https://list1.mysteel.com/market/p-221-----01010302-0-010101-------1.html'),
 ('上海市场镀锌板卷',
  'https://list1.mysteel.com/market/p-221-----01010501-0-010101-------1.html'),
 ('上海市场无缝管',
  'https://list1.mysteel.com/market/p-221-----01010901-0-010101-------1.html'),
 ('沈阳市场H型钢',
  'https://list1.mysteel.com/market/p-223-----01010705-0-01050101-------1.html'),
 ('上海市场方矩管',
  'https://list1.mysteel.com/market/p-221-----01010904-0-010101-------1.html'),
 ('长春市场工角槽钢',
  'https://list1.mysteel.com/market/p-223-----01010701-0-01050201-------1.html'),
 ('长春市场冷轧板卷',
  'https://list1.mysteel.com/market/p-223-----01010401-0-01050201-------1.html'),
 ('烟台市场热轧开平板',
  'https://list1.mysteel.com/market/p-223

In [160]:
def datetime_str_to_date (date_time_str):
    """
    convert dateimte str to dateimte to date
    input '%Y-%m-%d %H:%M'
    output %Y-%m-%d
    """
    try:
        date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M')
        date_ojb = date_time_obj.date()
    except TypeError:
        print('Type error!')
    else:
        return date_ojb

In [161]:
#没用上
def get_n_pages_url (page1_url, n):
    """
    input first page url ending with '1.html'
    output list of url of n pages
    """
    page_n_list = []
    for i in range(1, n + 1):
        page_n_url = page1_url.replace('1.html', str(i) + '.html')
        page_n_list.append(page_n_url)
    return page_n_list

In [162]:
def get_title_info (input_page_list):
    """
    input a list of page url
    output a list of tile info
    """
    page_list = []
    for url in input_page_list:
        response = requests.get(url)
        content = response.content
        soup = BeautifulSoup(content, "lxml")

        all_titles = soup.findAll('li', {"data-val":"0"})

        for i in range(len(all_titles)):
            txt = all_titles[i]
            url = 'https:' + txt.a.get('href')
            title = txt.a.get('title')
            date = datetime.datetime.strptime(txt.span.get_text(), '%Y-%m-%d %H:%M')
            page_list.append([title, date, url])

    return page_list

In [163]:
 """dict1"""
# d1_item = dict_urls_1_list[4]
# pages = get_n_pages_url(d1_item[1], n = page_range)
# titles = get_title_info(pages)
# title_list_filtered = []
# for i in range(len(titles)):
#     if d1_item[0] in titles[i][0]:
#         title_list_filtered.append(titles[i])
# print(Fore.YELLOW, d1_item) #print title
# df_list = []
# for item in title_list_filtered:
#     print(item[0])
#     title = item[0]
#     date = item[1]
#     url = item[2]

#     response = requests.get(url, headers = headers)
#     content = response.text
#     soup = BeautifulSoup(content,'lxml')

#     table_header = soup.findAll('th')
#     header_list = []
#     for i in table_header:
#         text = i.get_text()
#         text_strip = text.strip('\r\n\t请筛选').strip()
#         header_list.append(text_strip)

#     table_body = soup.find('tbody')
#     table_row = table_body.findAll('tr')
#     for tr in table_row:
#         td = tr.findAll('td')
#         row_list = []
#         for i in td:
#             text = i.get_text()
#             text_strip = text.strip('\r\n\t请筛选').strip()
#             row_list.append(text_strip)
#         df_list.append([date,title, url] + row_list[:7])

# df = pd.DataFrame(data = df_list, columns = ['日期','标题','网址'] + header_list[:7]) #保留3+7列
# df = df[(df['日期'] >= str(start_date)) & (df['日期'] < str(end_date))]
# df = df.astype({str(np.array(df.columns)[df.columns.str.contains(pat = '价格')][0]): int}) #价格列数据类型转换为数值
# print(Fore.BLACK, df)

'dict1'

In [164]:
"""dict2"""
d2_item = dict_urls_2_list[10]
pages = get_n_pages_url(d2_item[1], n = page_range)
titles = get_title_info(pages)
title_list_filtered = []
for i in range(len(titles)):
    if d2_item[0] in titles[i][0]:
        title_list_filtered.append(titles[i])
print(Fore.YELLOW, d2_item) #print title
df_list = []
for item in title_list_filtered:
    print(item[1])
    title = item[0]
    date = item[1]
    url = item[2]

    response = requests.get(url, headers = headers)
    content = response.text
    soup = BeautifulSoup(content,'lxml')

    table_body = soup.find('table',{'id':'marketTable'})
    table_row = table_body.findAll('tr')
    table_list=[]
    for tr in table_row:
        td = tr.findAll('td')
        row_list = []
        for i in td:
            text = i.get_text()
            text_strip = text.strip('\r\n\t请筛选').strip()
            row_list.append(text_strip)
#         df_list.append([date,title, url] + row_list[:7])
        table_list.append([date,title, url] + row_list[:7])
    header_list = table_list.pop(0) #剪切第一列‘筛选’
    table_list.pop(0)
    df_list.extend(table_list)
df = pd.DataFrame(data = df_list, columns = ['日期','标题','网址'] + header_list[3:10]) #保留3+7列
# df = df[(df['日期'] >= str(start_date)) & (df['日期'] < str(end_date))]
df = df.astype({str(np.array(df.columns)[df.columns.str.contains(pat = '价格')][0]): int}) #价格列数据类型转换为数值
print(Fore.BLACK, df)

 ('烟台市场热轧开平板', 'https://list1.mysteel.com/market/p-223-----01010306-0-01010205-------1.html')
2021-04-09 10:40:00
2021-04-08 10:36:00
2021-04-07 10:58:00
2021-04-06 11:09:00
2021-04-02 11:06:00
2021-04-01 10:35:00
2021-03-31 10:59:00
2021-03-30 11:05:00
2021-03-29 11:10:00
2021-03-26 11:07:00
2021-03-25 16:00:00
2021-03-25 10:42:00
2021-03-24 11:05:00
2021-03-23 11:19:00
2021-03-22 11:23:00
2021-03-19 11:08:00
2021-03-18 10:28:00
2021-03-17 10:20:00
2021-03-16 10:31:00
2021-03-15 10:38:00
2021-03-12 16:17:00
2021-03-12 11:07:00
2021-03-11 10:30:00
2021-03-10 10:52:00
2021-03-09 16:06:00
2021-03-09 10:34:00
2021-03-08 11:04:00
2021-03-05 10:46:00
2021-03-04 10:53:00
2021-03-03 11:12:00
2021-03-02 10:39:00
2021-03-01 10:48:00
2021-02-26 11:06:00
2021-02-25 10:22:00
2021-02-24 10:33:00
2021-02-23 10:21:00
2021-02-22 11:04:00
2021-02-20 10:43:00
2021-02-19 16:28:00
2021-02-19 11:00:00
2021-02-18 11:00:00
2021-02-10 09:18:00
2021-02-09 09:10:00
2021-02-08 10:06:00
2021-02-07 09:59:00
2021-0

In [165]:
writer = pd.ExcelWriter(local_path, mode = 'a', engine = 'openpyxl') #一个excel中的第一个sheet用'w'，其余用'a'
df.to_excel(writer, index = False, header = True, sheet_name = d2_item[0])
writer.save()
writer.close()